In [1]:
from folium import Map, PolyLine
import json

try:
    from routing_module import *
except:
    import sys
    sys.path.append('/home/happy956/')
    from routing_module import *

### 1. shuttle bus station

In [2]:
data = [
    {
        'start_station': '양촌역',
        'end_station': '더럭스나인',
        'start_lat': 37.641973,
        'start_lon': 126.614308,
        'end_lat': 37.644226,
        'end_lon': 126.618597,
        'direction': 1
    },
    {
        'start_station': '더럭스나인',
        'end_station': '구래역',
        'start_lat': 37.644226,
        'start_lon': 126.618597,
        'end_lat': 37.644496,
        'end_lon': 126.627346,
        'direction': 1
    },
    {
        'start_station': '구래역',
        'end_station': '디원시티',
        'start_lat': 37.644496,
        'start_lon': 126.627346,
        'end_lat': 37.643415,
        'end_lon': 126.618366,
        'direction': 2
    },
    {
        'start_station': '디원시티',
        'end_station': '양촌역',
        'start_lat': 37.643415,
        'start_lon': 126.618366,
        'end_lat': 37.641856,
        'end_lon': 126.614282,
        'direction': 2
    },
]

In [3]:
with open('data/shuttle_bus/shuttle_bus_station.json', 'w', encoding='utf8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

### 2. shuttle bus path

In [14]:
def get_total_path(data, direction):
    total_routes = []
    first, second = data[0], data[1]
    
    if direction == 'gurae':
        start_lon, start_lat = first['start_lon'], first['start_lat']
        end_lon, end_lat = first['end_lon'], first['end_lat']
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes
        
        start_lon, start_lat = second['start_lon'], second['start_lat']
        end_lon, end_lat = [126.619992, 37.644998]
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes[1:]
        
        start_lon, start_lat = [126.619992, 37.644998]
        end_lon, end_lat = second['end_lon'], second['end_lat']
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes[1:]
    
    elif direction == 'yongchon':
        start_lon, start_lat = first['start_lon'], first['start_lat']
        end_lon, end_lat = [126.627718, 37.643462]
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes
        
        start_lon, start_lat = [126.627718, 37.643462]
        end_lon, end_lat = [126.621685, 37.639407]
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes[1:]
        
        start_lon, start_lat = [126.621685, 37.639407]
        end_lon, end_lat = first['end_lon'], first['end_lat']
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes[1:]
        
        start_lon, start_lat = second['start_lon'], second['start_lat']
        end_lon, end_lat = second['end_lon'], second['end_lat']
        routes, _, _ = osrm_get_route((start_lon, start_lat, end_lon, end_lat))
        total_routes += routes[1:]
    
    return total_routes

In [15]:
with open('data/shuttle_bus/shuttle_bus_station.json', 'r', encoding='utf8') as f:
    data = json.load(f)

gurae_direction = list(filter(lambda x: x['direction'] == 1, data))
yongchon_direction = list(filter(lambda x: x['direction'] == 2, data))

In [16]:
m = Map(location=[37.643, 126.617], zoom_start=16)

gurae_direction_path = get_total_path(gurae_direction, 'gurae')
PolyLine(locations=gurae_direction_path, color='blue').add_to(m)

yongchon_direction_path = get_total_path(yongchon_direction, 'yongchon')
PolyLine(locations=yongchon_direction_path, color='red').add_to(m)

m    

93.0
57.0
81.60000000000001


In [7]:
with open('data/shuttle_bus/shuttle_bus_path.json', 'w') as f:
    total_path = [
        {
            'direction': 1,
            'route': list(map(lambda x: x[::-1], gurae_direction_path)),
        },
        {
            'direction': 2,
            'route': list(map(lambda x: x[::-1], yongchon_direction_path)),
        },
    ]
    json.dump(total_path, f)

### 3. extract trip & timestamp

In [17]:
with open('data/shuttle_bus/shuttle_bus_path.json', 'r') as f:
    path = json.load(f)

In [19]:
gurae_direction_path = list(filter(lambda x: x['direction'] == 1, path))[0]
yongchon_direction_path = list(filter(lambda x: x['direction'] == 2, path))[0]

In [44]:
## gurae direction
gurae_trip_data = []
yongchon_trip_data = []

for idx, t in enumerate(np.arange(420, 480 + 55 + 1, 5)):
    routes = gurae_direction_path['route']
    timestamp = get_timestamp(routes, t, 7)
    timestamp = [float(x) for x in timestamp]
    gurae_trip_data.append({
        'id': idx,
        'trip': routes,
        'timestamp': timestamp,
        'direction': 1
    })

for idx, t in enumerate(np.arange(420, 480 + 55 + 1, 5)):
    routes = yongchon_direction_path['route']
    timestamp = get_timestamp(routes, t, 7)
    timestamp = [float(x) for x in timestamp]
    yongchon_trip_data.append({
        'id': idx,
        'trip': routes,
        'timestamp': timestamp,
        'direction': 2
    })

with open('data/shuttle_bus/shuttle_bus_trip.json', 'w') as f:
    total_trip = gurae_trip_data + yongchon_trip_data
    # display(total_trip)
    json.dump(total_trip, f)